In [1]:
import torch
import torchvision
import torchdata
from ceruleanml import plot
import random
from ceruleanml import evaluation
import warnings
import sys

if not sys.warnoptions:
    warnings.simplefilter("ignore")
# Set the random seed
seed=0 # we need to set this for torch datapipe separately
random.seed(seed)

In [2]:
model_path = "/slickformer/data/models/2023_02_18_00_38_07_4cls_rn152_pr512_px1024_1440min_maskrcnn_scripting_cpu_model.pt"
scripted_model = torch.jit.load(model_path)

In [3]:
test_scenes = ["../data/tile_with_slick_512_512_3band.png"] # from cerulean cloud test fixture

In [4]:
from ceruleanml.data_pipeline import put_image_in_dict, channel_first_norm_to_tensor
source_pipe = torchdata.datapipes.iter.IterableWrapper(iterable=test_scenes)

source = (source_pipe 
    .read_tiff()
    .map(put_image_in_dict)
    .map(channel_first_norm_to_tensor)
)

In [5]:
next(iter(source))['image'].shape

torch.Size([3, 512, 512])

In [6]:
# time is an issue where there are lots of slicks
from ceruleanml.inference import mrcnn_3_class_inference
from ceruleanml.data_creation import class_dict

bbox_conf_threshold = .2
mask_conf_threshold=.2
input_size=2000
for idict in source:
    pred_dict_thresholded, pred_dict = mrcnn_3_class_inference([idict['image']], scripted_model, bbox_conf_threshold, mask_conf_threshold, input_size=input_size)
    pred_dict_thresholded_nms , pred_dict = mrcnn_3_class_inference([idict['image']], scripted_model, bbox_conf_threshold, mask_conf_threshold, input_size=input_size, interclass_nms_threshold=.5)

In [61]:
pred_dict_thresholded_nms

{'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
 'boxes': tensor([[  2., 114., 154., 221.]]),
 'scores': tensor([0.8743]),
 'labels': tensor([3])}

In [ ]:
from torchmetrics import detection

m = detection.mean_ap.MeanAveragePrecision(box_format='xyxy', iou_type='bbox', iou_thresholds=[.5], rec_thresholds=None, max_detection_thresholds=None, class_metrics=True)

m.update(preds=[pred_dicts_thresholded_nms], target=[pred_dicts_thresholded_nms])

from pprint import pprint
pprint(m.compute())

{'map': tensor(1.),
 'map_50': tensor(1.),
 'map_75': tensor(-1),
 'map_large': tensor(1.),
 'map_medium': tensor(-1.),
 'map_per_class': tensor(1.),
 'map_small': tensor(-1.),
 'mar_1': tensor(1.),
 'mar_10': tensor(1.),
 'mar_100': tensor(1.),
 'mar_100_per_class': tensor(1.),
 'mar_large': tensor(1.),
 'mar_medium': tensor(-1.),
 'mar_small': tensor(-1.)}


In [62]:
import httpx
import configparser
import base64
# parses secrets so we don't expose them in the notebook
config = configparser.ConfigParser()
config.read('../.secrets')
config = config['secrets']
# reads image as bytes, converts bytes to string so it can be sent as a post request
with open(test_scenes[0], 'rb') as f:
    byte_string = f.read()
    base64_string = base64.b64encode(byte_string).decode('utf-8')
# we format the payload and headers according to what the docs expect
# docs are at f"{config['CERULEAN_STAGING_URL']}/docs"
predict_url = f"{config['CERULEAN_STAGING_URL']}/predict"
payload = {"stack": [{"image": base64_string, "bounds":[ 0,0,0,0 ]}]}
headers = {"Authorization": f"Bearer {config['CERULEAN_API_KEY']}"}
response = httpx.post(predict_url, json=payload, headers=headers, timeout=None)

In [59]:
response.json()

{'stack': [{'classes': None,
   'confidence': None,
   'bounds': [0.0, 0.0, 0.0, 0.0],
   'features': [{'type': 'Feature',
     'geometry': {'type': 'MultiPolygon',
      'coordinates': [[[[2.0, 110.0],
         [6.0, 110.0],
         [6.0, 111.0],
         [8.0, 111.0],
         [8.0, 112.0],
         [9.0, 112.0],
         [9.0, 116.0],
         [13.0, 116.0],
         [13.0, 117.0],
         [14.0, 117.0],
         [14.0, 118.0],
         [15.0, 118.0],
         [15.0, 120.0],
         [16.0, 120.0],
         [16.0, 122.0],
         [17.0, 122.0],
         [17.0, 123.0],
         [20.0, 123.0],
         [20.0, 124.0],
         [21.0, 124.0],
         [21.0, 125.0],
         [22.0, 125.0],
         [22.0, 126.0],
         [23.0, 126.0],
         [23.0, 127.0],
         [24.0, 127.0],
         [24.0, 128.0],
         [25.0, 128.0],
         [25.0, 129.0],
         [31.0, 129.0],
         [31.0, 130.0],
         [33.0, 130.0],
         [33.0, 131.0],
         [35.0, 131.0],
         [3

In [29]:
payload

{'stack': [{'image': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x02\x00\x08\x02\x00\x00\x00{\x1aC\xad\x00\x00 \x00IDATx\x9c\xec\xfd\xdb\xba\x1c7\xaen\x01\x82\x91:\xd8Uv\xd5\xf2^\xef\xff\xa0\xcbR\x06\xfb\x82\xe2\xf0\xe0\x8f\x90\xbb\xaf\xdb\x93\x17\xfaR1\x19$\x883@\x901~\xfb\xed\xb71\xc6\x18\xa3\xaa\xae\xeb\x9as\xbe\xdf\xef\xaa\x1ac\xbc^\xaf\xaa\x9as~\xfa\xf4\xe9\xfb\xf7\xef\xd5\xda\x9c\xf3\xbe\xef\xf5\xeejc\x8c\xfb\xbe\xd7\x8b\xd7uU\x15\x1d\xd6\x8f\xf5{\xce9\xc6\x98\xbb\xf1nU\xadw\xef\xfb\xbe\xef{=\xbf\xae\xeb\xbe\xef\xd5m\xfd\x95\xce\x0bZ\x03\xbf\xfez\xdf\xf7\x9a}\x8c\xf1~\xbf\xaf\xeb\xba\xaek\xcd\xe8\x0e\xc0\xb6\xc0XO\x0c\xf9zk\x8d@O\xd6\xbe^16\xc0\x18\xff\xbe\xdf\xef\xd7\xeb\xc5[L\xbaP\xb7\x1e\xb2\xa8\xd5\x8d\xe5\xac\xa9\x8d\xf0\xf5_\x0f\xe8\xe9\x18\x87\xb9\xd6\x9f\xc0\xc6z}=\x87j\x10\x82\x17\x17`k\xb4\xd7\xeb\xb5\xfe\x0b\xf6\x00\x9bI\x17\xd1\x8d\x99O\x9f>\xad\x87\xc6\xc3\x82\xe1\xba.~\xafa\xd7\x13C\xbe\xc0\x03\xff^f \n\x0et\xb7\xeb\xba>}\xfaTU\xdf\xbe}\x03Z3\t\xaf

In [60]:
pred_dicts_thresholded_nms

NameError: name 'pred_dicts_thresholded_nms' is not defined